https://www.kaggle.com/karthikbhandary2/boat-sales -- вот датасет с яхтами.

1) Постройте модель зависимости цены яхты от ее параметров. 

2) Насколько модель точна? (в качестве метрики можно выбрать mean absolute error)

3) Какие факторы в какой степени влияют на цену?

P.S. Если вам покажется, что яхты это скучно, вы можете взять любой другой датасет с kaggle)